In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [3]:
df = spark.read.csv("D:\\Udemy course\\Spark\\MLLib\\cruise_ship_info.csv", inferSchema=True, header=True)
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [4]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [7]:
from pyspark.sql.functions import corr
df.select(corr('crew','cabins'),
         corr('crew','passengers')).show()

+------------------+----------------------+
|corr(crew, cabins)|corr(crew, passengers)|
+------------------+----------------------+
|0.9508226063578497|    0.9152341306065384|
+------------------+----------------------+



In [13]:
#checking out the categorical column
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [8]:
#converting categorical column into numeric column
from pyspark.ml.feature import StringIndexer
new_df = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_cat')
new_df = new_df.fit(df).transform(df)

In [9]:
new_df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [14]:
new_df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_cat']

In [27]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['Age','Tonnage','passengers', 'length','cabins','passenger_density','Cruise_line_cat'],
                            outputCol = 'features')
output = assembler.transform(new_df)
for p in output.head(5):
    print(p)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_line_cat=1.0, features=DenseVector([26.0, 47.262, 14.86, 7.22, 7.43, 31.8, 1.0]))


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, Cruise_line_cat=1.0, features=DenseVector([11.0, 110.0, 29.74, 9.53, 14.88, 36.99, 1.0]))


Ro

In [28]:
final_data = output.select(['features','crew'])
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [29]:
train, test = final_data.randomSplit([0.8,0.2])

In [31]:
#applying the model and evaluating
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol = 'crew')
lr_model = lr.fit(train)
test_results = lr_model.evaluate(test)
print("Test accuracy: {0}".format(test_results.r2))
print("Test rmse    : {0}".format(test_results.rootMeanSquaredError))

Test accuracy: 0.978732848696514
Test rmse    : 0.5539723442994113


In [33]:
#predicted values
pred = lr_model.transform(test)
pred.show()

+--------------------+----+------------------+
|            features|crew|        prediction|
+--------------------+----+------------------+
|[4.0,220.0,54.0,1...|21.0|20.890084330242512|
|[5.0,122.0,28.5,1...| 6.7| 5.666299734300644|
|[6.0,110.23899999...|11.5|11.116462282315974|
|[6.0,112.0,38.0,9...|10.9|11.285035053804874|
|[8.0,110.0,29.74,...|11.6|12.187058843821116|
|[9.0,116.0,26.0,9...|11.0|11.102380293500707|
|[10.0,86.0,21.14,...| 9.2| 9.704228653060985|
|[12.0,25.0,3.88,5...|2.87|3.0861245985666828|
|[12.0,90.09,25.01...|8.68|  8.80741490217894|
|[12.0,91.0,20.32,...|9.99| 9.158005745870911|
|[15.0,30.27699999...| 4.0|3.8930465931222415|
|[15.0,70.367,20.5...| 9.2| 8.703615119997758|
|[16.0,77.499,19.5...| 9.0|  9.20563299631986|
|[16.0,77.71300000...|9.09|  8.49922319777494|
|[16.0,78.491,24.3...|7.65|  8.16447179435649|
|[18.0,69.153,18.8...|7.94| 8.341255949659287|
|[20.0,50.76,17.48...|6.14| 7.408365004010724|
|[21.0,10.0,2.08,4...| 1.6|1.7178288634717478|
|[21.0,19.093